# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
df = pd.read_csv('main_task.xls')

In [3]:
# Функции

def find_date1(x):
    '''Функция возвращает первое значение из строки с датой'''
    pattern = re.compile('\d\d/\d\d/\d\d\d\d')
    date1 = pattern.search(x)
    if date1 != None:
        return date1.group(0)
        
def find_date2(x):
    '''Функция возвращает последнее значение из строки с датами'''
    pattern = re.compile('\d\d/\d\d/\d\d\d\d')
    date2 = pattern.findall(x)
    if len(date2)>1:
        return date2[-1]

In [4]:
# Заполняем пропуски в значениях
df['Number of Reviews'] = df['Number of Reviews'].fillna(0)
df['Price Range'].fillna(df['Price Range'].mode()[0], inplace=True)

# Вместо пропусков в количестве кухонь считаем, что ресторан представлен одной кухней 
df['Cuisine Style'] = df['Cuisine Style'].fillna("['Other']")

In [5]:
# Создаем столбец с количеством кухонь
df['Number of Cuisine Styles'] = df['Cuisine Style'].apply(lambda x: len(list(x[1:-1].split(', '))))

In [6]:
# Создаем новые столбцы с датами отзывов
df['Review date1'] = df.Reviews.apply(find_date1)
df['Review date1'] = pd.to_datetime(df['Review date1'])

df['Review date2'] = df.Reviews.apply(find_date2)
df['Review date2'] = pd.to_datetime(df['Review date2'])

# Создаем столбец с количеством дней между отзывами
df['Review date diff'] = df['Review date1'] - df['Review date2']
df['Review date diff'] = df['Review date diff'].apply(lambda x: abs(x.days))

# Создаем столбец с количеством дней после последнего отзыва
from datetime import date
df['From last review'] = datetime.today() - df['Review date1']
df['From last review'] = df['From last review'].apply(lambda x: x.days)

# Заполняем пропуски медианным значением в колонках с разницей дней
df['Review date diff'].fillna(df['Review date diff'].median(), inplace=True)
df['From last review'].fillna(df['From last review'].median(), inplace=True)


In [7]:
# Переводим столбец с ценовой категорией в числовой формат
price_dict = { '$$$$':2,
               '$$ - $$$':1, 
               '$':0}
df['Price Range'] = df['Price Range'].replace(to_replace=price_dict)

In [8]:
# Добавляем колонки с площадью города и количеством населения
city_dict = {}
data = [[2.2, 105], [1, 188], [8.9, 1706], 
         [3.6, 891], [1.5, 310], [0.3, 41.4], 
         [1.4, 181], [0.4, 368], [1.9, 414], 
         [2.9, 1287], [1.7, 101], [3.3, 607], 
         [1.2, 117], [0.2, 32.6], [0.4, 88], 
         [1.8, 517], [1.8, 525], [0.8, 86.4], 
         [0.9, 219], [0.5, 48], [1.9, 755], 
         [0.5,100], [1.3, 496], [0.7, 454], 
         [0.7,213], [0.5, 120], [0.2, 16], 
         [0.3, 163], [0.7, 39], [0.125, 51.5], 
         [0.8, 327]]
cities = df.City.unique()
for i in range(len(cities)):
    city_dict[cities[i]] = data[i] 
    
df['City square'] = df['City'].apply(lambda x: city_dict[x][0])
df['City population'] = df['City'].apply(lambda x: city_dict[x][1])

In [9]:
# Анализируем степень корреляции переменных между собой и Rating
df.corr()

,Ranking,Rating,Price Range,Number of Reviews,Number of Cuisine Styles,Review date diff,From last review,City square,City population
Ranking,1.000000,-0.368371,-0.026525,-0.222072,-0.320250,0.091467,0.166834,0.570810,0.404542
Rating,-0.368371,1.000000,-0.028588,0.026924,0.120059,-0.070799,-0.087124,-0.024219,0.017983
Price Range,-0.026525,-0.028588,1.000000,0.107162,0.003177,-0.030718,0.001173,-0.018437,-0.038465
Number of Reviews,-0.222072,0.026924,0.107162,1.000000,0.405641,-0.125437,-0.174350,0.023761,0.047716
Number of Cuisine Styles,-0.320250,0.120059,0.003177,0.405641,1.000000,-0.184014,-0.280628,0.046756,0.079100
Review date diff,0.091467,-0.070799,-0.030718,-0.125437,-0.184014,1.000000,0.141528,-0.043083,-0.041571
From last review,0.166834,-0.087124,0.001173,-0.174350,-0.280628,0.141528,1.000000,-0.043386,-0.044541
City square,0.570810,-0.024219,-0.018437,0.023761,0.046756,-0.043083,-0.043386,1.000000,0.911371
City population,0.404542,0.017983,-0.038465,0.047716,0.079100,-0.041571,-0.044541,0.911371,1.000000


In [10]:
# Количество населения и площадь сильно коррелируют между собой,
# поэтому введем признак, обозначающий плотность населения, 
# а столбцы с количеством населения и площадью удалим.
df['Population density'] = df['City population'] / df['City square']

In [11]:
# Преобразуем колонки City и Cuisine Style, исполязуя dummy-переменные
# Добавим столбцы с dummy-переменными к датафрейму
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x[1:-1])
df_cuisine = df['Cuisine Style'].str.get_dummies(sep=', ')
df_city = pd.get_dummies(df.City)
df = pd.concat([df, df_cuisine], axis=1)
df = pd.concat([df, df_city], axis=1)

In [12]:
# Удаляем столбцы с категориальными переменными
df = df.drop(['City', 'Restaurant_id', 'Cuisine Style', 'Price Range', 
              'Reviews', 'URL_TA', 'ID_TA', 'Review date1', 
              'Review date2', 'City population', 'City square'
             ], axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [13]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [14]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [15]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [16]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [17]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [18]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20564000000000002
